# Futures

Try out a moving average crossover strategy on futures.  One at a time and summarize the results

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *

import pinkfish as pf
import strategy

#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Investment Universe:  

Futures contracts cover these main asset classes:  
- Currencies
- Energies
- Financials
- Grains
- Indices
- Meats
- Metals
- Softs

### FUTURES
https://www.barchart.com/futures/contract-specifications/currencies

In [3]:
# symbol: (description, multiplier)

currencies = {
    'DX=F':  ('U.S. Dollar Index', 1000),
    'BTC=F': ('Bitcoin Futures', 5),
    '6B=F':  ('British Pound', 62500),
    '6C=F':  ('Canadian Dollar', 100000),
    '6J=F':  ('Japanese Yen', 125000),
    '6S=F':  ('Swiss Franc', 125000),
    '6E=F':  ('Euro FX', 125000),
    '6A=F':  ('Australian Dollar', 100000),
    '6M=F':  ('Mexican Peso', 500000),
    '6N=F':  ('New Zealand Dollar', 100000),
    '6Z=F':  ('South African Rand', 500000),
    '6L=F':  ('Brazilian Real', 100000),
    '6R=F':  ('Russian Ruble', 500000)  
}

energies = {
    'CL=F':  ('Crude Oil West Texas Intermediate', 1000),
    'HO=F':  ('New York Harbor ULSD', 42000),
    'RB=F':  ('Gasoline Blendstock New York Harbor [RBOB]', 42000),
    'NG=F':  ('Natural Gas', 10000),
    #'BZ=F':  ('Brent Crude Oil Financial Futures', 1000),
    'EH=F':  ('Ethanol', 29000)
}

financials = {
    'ZB=F':  ('U.S. Treasury Bond Futures', 1000),
    'UB=F':  ('Ultra Treasury Bond', 1000),
    'ZN=F':  ('10-Year Treasury-Note', 1000),
    'TN=F':  ('Ultra 10-Year Treasury-Note', 1000),
    'ZF=F':  ('5-Year Treasury-Note', 1000),
    'ZT=F':  ('2-Year Treasury-Note', 2000),
    'ZQ=F':  ('30-Day Fed Funds', 2000),
    #'GE=F':  ('EuroDollar', 2500)
}

grains = {
    'ZC=F':  ('Corn', 50),
    'ZS=F':  ('Soybean', 50),
    'ZM=F':  ('Soybean Meal', 1000),
    'ZL=F':  ('Soybean Oil', 600),
    'ZW=F':  ('Chicago Soft Red Winter Wheat', 50),
    'KE=F':  ('KC Hard Red Winter Wheat', 50),
    'ZO=F':  ('Oats', 50),
    #'ZR=F':  ('Rough Rice', 2000),
    'ZS=F':  ('Rapeseed Canola', 20)
}

indices = {
    'ES=F':  ('E-Mini S&P 500 Index', 50),
    'NQ=F':  ('E-Mini Nasdaq 100', 20),
    'YM=F':  ('E-Mini Dow Jones Industrial Averagen', 5),
    'RTY=F': ('E-Mini Russell 2000 Index', 50),
    #'VI=F':  ('CBOE Volatilty Index VIX Futures', 1000),
    #'GD=F':  ('GSCI - Goldman Sachs Commodity Index', 250)
}

meats = {
    'LE=F':  ('Live Cattle', 400),
    'GF=F':  ('Feeder Cattle', 500),
    'HE=F':  ('Lean Hogs', 400),
    #'KM=F':  ('Pork Cutout', 400),
    'DC=F':  ('Milk Class III', 2000)
}

metals = {
    'GC=F':  ('Gold 100-oz', 100),
    'SI=F':  ('Silver 5,000-oz', 5000),
    'HG=F':  ('High Grade Copper', 25000),
    'PL=F':  ('Platinum', 50),
    'PA=F':  ('Palladium', 100),
}

softs = {
    'CT=F':  ('Cotton #2', 500),
    #'OJ=F':  ('Orange Juice [FCOJ-A]', 150),
    'KC=F':  ('Coffee C Arabica', 375),
    'SB=F':  ('Sugar #11', 1120),
    'CC=F':  ('Cocoa', 10),
    'LBS=F': ('Lumber', 110),
    'SF=F':  ('Sugar #16', 1120)
}

merged = {**currencies, **energies, **financials, **grains,
          **indices, **meats, **metals, **softs}

Globals

In [4]:
symbols = list(softs)
#symbols = ['ES=F', 'GC=F', 'CL=F']
capital = 100000
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

# set options
stop_loss_pct = 0/100
margin = 1
sma_fast = 10
sma_slow = 100
percent_band = 0/100
enable_shorts = True

Run Strategy

In [5]:
strategies = pd.Series(dtype=object)
for symbol in symbols:
    print("{0}".format(symbol), end=" ")

    strategies[symbol] = strategy.Strategy(symbol, capital, start, end)
    
    # set options
    strategies[symbol].stop_loss_pct = stop_loss_pct
    strategies[symbol].margin = margin
    strategies[symbol].multiplier = merged[symbol][1]
    strategies[symbol].timeperiod_fast = sma_fast
    strategies[symbol].timeperiod_slow = sma_slow
    strategies[symbol].percent_band = percent_band
    strategies[symbol].enable_shorts = enable_shorts

    #run
    strategies[symbol].run()
    
    #get logs
    _, strategies[symbol].tlog, strategies[symbol].dbal = strategies[symbol].get_logs()
    strategies[symbol].stats = strategies[symbol].get_stats()
#strategies[symbol].tlog.head(50)

CT=F KC=F SB=F CC=F LBS=F SF=F 

Summarize results

In [6]:
metrics = ('start',
           'annual_return_rate',
           'max_closed_out_drawdown',
           'sharpe_ratio',
           'sortino_ratio',
           'monthly_std',
           'annual_std',
           'pct_time_in_market',
           'total_num_trades',
           'pct_profitable_trades',
           'avg_points',
           'ending_balance')

df = strategy.summary(strategies, metrics)
pd.set_option('display.max_columns', len(df.columns))
df

,CT=F,KC=F,SB=F,CC=F,LBS=F,SF=F
start,2000-05-02,2000-05-12,2000-07-21,2000-05-09,2000-12-05,2000-08-03
annual_return_rate,5.15,-5.99,-0.33,-9.58,-5.87,-100.00
max_closed_out_drawdown,-56.21,-77.17,-63.87,-87.25,-75.99,-1235.91
sharpe_ratio,0.28,-0.04,0.13,-0.51,-0.24,-0.22
sortino_ratio,0.64,-0.07,0.26,-0.36,-0.26,-0.15
monthly_std,8.28,10.75,10.75,4.95,5.22,83.36
annual_std,28.25,32.07,43.70,17.64,18.80,89.63
pct_time_in_market,100.00,68.68,100.00,49.14,92.28,99.92
total_num_trades,79,75,89,60,92,103
pct_profitable_trades,31.65,25.33,29.21,20.00,29.35,27.18


In [7]:
# averages
avg_annual_return_rate = df.loc['annual_return_rate'].mean()
avg_sharpe_ratio = df.loc['sharpe_ratio'].mean()
print('avg_annual_return_rate: {}'.format(avg_annual_return_rate))
print('avg_sharpe_ratio:       {}'.format(avg_sharpe_ratio))

avg_annual_return_rate: -19.437664108360483
avg_sharpe_ratio:       -0.09963928796771011
